In [116]:
import pandas as pd
import numpy as np
import os
import re

import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/IW_student/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [117]:
data=pd.read_csv('~/work/food_flavor/kochbar_ingredient_preprocessing/datasets/long_ingr_data/long_ingr_trans_003.csv',index_col=0)
data.shape

(7098, 5)

In [118]:
two_ingr=data[data['en_ingr'].str.split().str.len()==2]
two_ingr.shape

(1956, 5)

In [4]:
two_ingr.head()

,recipe_id,recipe_href,name,ingredient_name,en_ingr
2700,11059,/rezept/130203/Dr-Bello-s-Frikadellos.html,pfeffer aus der mühle weiss,weiss Pfeffer,white pepper
2701,2761,/rezept/478153/Auberginenroellchen-mit-Kaese.html,evtl. schwarze oliven zumgarnieren,schwarze oliven,black olives
2702,55182,/rezept/404267/Krabbenruehrei.html,pfeffer aus der mühle bunt,bunt Pfeffer,variegated pepper
2703,94189,/rezept/351602/Hokaido-Eintopf.html,chilli rot grob gehackt,Chili rot,chili rot
2704,15844,/rezept/392070/Fleisch-Rind-Koefte-an-Zitronen...,chilischoten piri-pirigetrocknet im mörserzers...,Peperoncino,chili pepper


In [5]:
# change "Chili con carne"

In [5]:
two_ingr['en_ingr'][two_ingr['ingredient_name']=='Chili con carne']='Chili_con_carne'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/pandas/core/series.py:1020: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [6]:
two_ingr_list=list(set(two_ingr['en_ingr'].values.tolist()))
len(two_ingr_list)

842

In [7]:
two_ingr_list

['flour water',
 'tomato pepper',
 'olive oil',
 'sugar start',
 'herbs salt',
 'limettenscheiben reintun',
 'thyme leaves',
 'prawns shell',
 'mushrooms zbsteinpilze',
 'cherry tomatoes',
 'pineapple juice',
 'suitable alcohol',
 'spaetzle kind',
 'grilled peppers',
 'fruit crispies',
 'clove garlic',
 'ras hanout',
 'sausages crackers',
 'red cabbage',
 'emmental cheese',
 'soup noodles',
 'date tomatoes',
 'ital camembert',
 'pesto verde',
 'soup green',
 'turkey schnitzel',
 'heart caramels',
 'wild garlic',
 'flower salt',
 'weisser pfefferadmühle',
 'oranges back',
 'lime juice',
 'green pepper',
 'red lollo',
 'oil schnitzel',
 'roll x22rolo',
 'biskin special',
 'sauce powder',
 'season herbs',
 'glasses á',
 'tuna juice',
 'beetroot ballsà',
 'tomato mitkräutern',
 'black pudding',
 'spicy woodpecker',
 'seasoned spices',
 'lake cucumbers',
 'base powder',
 'beetroot balls',
 'baiser doesn39t',
 'medium shrimp',
 'bass net',
 'oil rice',
 'pul biber',
 'chocolate bases',
 'red

In [ ]:
# ingredients which could be keep direclty

In [6]:
keep_ingr=['alba oil','anaheim chili','anchovy paste','apple jelly','arrowroot starch','asia spice', 'baked potato',
           'bamboo shoots','barbeque spice','barley barley','base powder','bay leaf','beef broth','black olives',
          'black pepper','black tapenade','blood oranges','brown sugar','brussels sprouts','caraway seed',
          'cherry tomatoes','corn starch','cream cheese','edamer cheese','emmental cheese','french beans',
          'green olives','green peas','hemp flour','hot chutney','hot sauce','ice cream','italian herbs',
          'juniper berry','olive oil','orange juice','oyster mushrooms','paprika powder','parmesan cheese',
          'pineapple juice','red beans','sheep cheese','sour cream','sour cucumbers','spicy cheese','sunflower oil',
          'tomato basilikum','vegetable oil','white beans','white pepper','white wine']

In [7]:
two_ingr_done=two_ingr[two_ingr['en_ingr'].isin(keep_ingr)]
two_ingr_done.shape

(513, 5)

In [8]:
two_ingr_pending=two_ingr[~two_ingr['en_ingr'].isin(keep_ingr)]
two_ingr_pending.shape

(1443, 5)

In [9]:
two_ingr_pending.reset_index(drop=True,inplace=True)
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].str.strip()
two_ingr_pending=two_ingr_pending[two_ingr_pending['en_ingr']!='']
two_ingr_pending.dropna(inplace=True)
two_ingr_pending.reset_index(drop=True,inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
two_ingr_pending.shape

(1443, 5)

In [11]:
two_ingr_pending.tail()

,recipe_id,recipe_href,name,ingredient_name,en_ingr
1438,95463,/rezept/257576/Pasta-Tortellini-mit-Weisswein-...,tortellini gefüllt mitkäse - wenn´s eilig ist ...,tortellini gefüllt mitkäse,tortellini cheese
1439,27100,/rezept/259314/FLEISCH-Osso-Buco-a-la-Emi.html,kalbshaxe in dicken scheiben,kalbshaxe in dicken scheiben,knuckle veal
1440,61424,/rezept/78521/3-C-Suppe.html,chili con carne instant z.b.knorr,Chili con carne,Chili_con_carne
1441,26194,/rezept/264389/Berliner-Brot.html,mit schale gemahlene mandeln,mit schale gemahlene mandeln,almonds shell
1442,26194,/rezept/264389/Berliner-Brot.html,ein paar tropfen warmeswasser,ein paar tropfen warmeswasser,warm water


In [ ]:
# delete some words

In [12]:
del_ingrs=['decoration','decorate','spoon','sprig','spring','sprinkled','sprinkles','st','simply','stalk','stalks',
         'stripped','strong','stuffed','decor','instructions','liking','mixture','start','suitable','sundried',
         'sweetness','teaspoons','stores','various','color','wild','bright','classic','garnish','stock','hard',
         'heart','light','lowfat','matter','goes','today','rose','rotten','theaten','topside','powdered','pul',
         'inside','unpeeled','seasonal','season','seedless','semifat','shredded','vacuum','smkb','bulp','variegated',
          'knuckle']

In [13]:
def del_ingr(ingredient):
    no_uningr=' '.join([c for c in ingredient.split() if c not in del_ingrs])
    return no_uningr

In [14]:
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:del_ingr(x))

In [ ]:
# pulp removed, sense touch,smkb brushing,syringe needle,stone crush

In [15]:
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('pulp removed',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('sense touch',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('smkb brushing',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('syringe needle',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('stone crush',''))

In [ ]:
# mushroom herb-->Pilzkraut, noilly prat-->Noilly_Prat,nuremberg sausages-->Nürnberger_Bratwurst, 
# onke stichfest-->Onke_Stichfest, passed wounds-->anchovy, patros mitkräutern-->Patros_mit_Kräutern,
# speckknödel-->Speckknödel, black forest--> pork, black pudding--> blood_sausage, 
# blutorange jogurth-->blood_orange_yogurt，bratwurst pork-->Bratwurst, breading flour-->breadcrumbs,
# brunch paprika-->Brunch_paprika,butcher onion--> Metzgerzwiebel,butter streusel-->Butterstreusel,
# cabbage sprouts-->brussels_sprouts,camembert garlic-->camembert_cheese, garlic,cherries morello-->cherry,
# chili carne--> Chili_con_carna, herbs derprovence-->Herbes_de_Provence,sultanas eyes-->raisin,
# ^don39t hack, oil foil,steif geschlagen, bread spice

In [16]:
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('mushroom herb','Pilzkraut'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('noilly prat','Noilly_Prat'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('nuremberg sausages','Nürnberger_Bratwurst'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('onke stichfest','Onke_Stichfest'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('passed wounds','anchovy'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('patros mitkräutern','Patros_mit_Kräutern'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('speckknödel','Speckknödel'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('black forest','pork'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('black pudding','blood_sausage'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('blutorange jogurth','blood_orange_yogurt'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('black forest','pork'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('bratwurst pork','Bratwurst'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('breading flour','breadcrumbs'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('brunch paprika','Brunch_paprika'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('butcher onion','Metzgerzwiebel'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('butter streusel','Butterstreusel'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('cabbage sprouts','brussels_sprouts'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('camembert garlic','camembert_cheese, garlic'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('cherries morello','cherry'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('chili carne','Chili_con_carne'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('herbs derprovence','Herbes_de_Provence'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('sultanas eyes','raisin'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('bouquet garni','Bouquet_garni'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('bacon noodle','Speckknödel'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('don39t hack',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('don39t need',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('doesn39t',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('oil foil',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('steif geschlagen',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('bread spice',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('x22',''))

In [17]:
two_ingr_pending[two_ingr_pending['en_ingr']=='tomato pepper']['ingredient_name'].values

array(['tomatenpaprika'], dtype=object)

In [18]:
two_ingr_pending.shape

(1443, 5)

In [19]:
pending_ingr=list(set(two_ingr_pending['en_ingr'].values.tolist()))
len(pending_ingr)

761

In [254]:
pending_ingr.sort()
pending_ingr

['Biskin_Spezial',
 'Blueberry',
 'Blumenkohl_Sud',
 'Boletus_erythropus',
 'Bolognese',
 'Bouquet_garni',
 'Bratwurst',
 'Brunch_paprika',
 'Butter_aroma',
 'Butterstreusel',
 'Camembert_cheese',
 'Chili_con_carne',
 'Cocktailtomato',
 'Curry_paste',
 'Foxys_vampirpaste',
 'Heichelheimer_Rohe_Kartoffelmasse',
 'Herbes_de_Provence',
 'Hirvenpotku',
 'Hörnchennudel',
 'Kräuter_der_Provence',
 'Kräuterbutter',
 'Kräutersalz',
 'Lebkuchenherz',
 'Liquid_from_the_pickled_cucumbers',
 'Lollo_rosso',
 'Loup_de_mer',
 'Mettwurst',
 'Metzgerzwiebel',
 'Mezzelune',
 'Nusskruste',
 'Nürnberger_Bratwurst',
 'Onke_Stichfest',
 'Patros_mit_Kräutern',
 'Pesto_alla_Genovese',
 'Pfanni_Knödel',
 'Pilzkraut',
 'Pimientos',
 'Pizza_e_Pasta',
 'Rama_Culinesse',
 'Ras_el_hanout',
 'Rot_Pfeffer',
 'Solperfleisch',
 'Sossenpulver',
 'Speckknödel',
 'Tafelspitz',
 'Tatare,knoblauch',
 'Tomato_al_gusto',
 'Vanille_Sossenpulver',
 'Würzcreme',
 'Würzspinat',
 'Zitronigfruchtiges_Curry',
 'alcohol',
 'almond',


In [18]:
keep_ingr=['white bean','whipped cream','wheat breads','vegetable mix','vegetables','yogurt sauce','yellow peppers','yellow pepper',
'white chocolate','white bean','vegetable broth','vanilla sugar','vanilla sauce', 'vanilla pudding','vanilla icecream',
'tomatoes juice','tomatoes gusto','tomato sauce','baking powder','beverage powder','cherry juice','cherry peppers',
'chilli cheese','chili sauce','chili powder','green beans','green pepper','green peppers','lemon juice',
'lemon zest','lime juice','red wine','smoked salmon','spice mix','spicy cheeses']

In [19]:
two_ingr_done_002=two_ingr_pending[two_ingr_pending['en_ingr'].isin(keep_ingr)]

In [20]:
two_ingr_done=pd.concat([two_ingr_done,two_ingr_done_002],axis=0,ignore_index=0)
two_ingr_done.shape

(600, 5)

In [21]:
two_ingr_pending=two_ingr_pending[~two_ingr_pending['en_ingr'].isin(keep_ingr)]
two_ingr_pending.shape

(1356, 5)

In [22]:
two_ingr_pending.reset_index(drop=True,inplace=True)

In [23]:
two_ingr_pending.tail()

,recipe_id,recipe_href,name,ingredient_name,en_ingr
1351,95463,/rezept/257576/Pasta-Tortellini-mit-Weisswein-...,tortellini gefüllt mitkäse - wenn´s eilig ist ...,tortellini gefüllt mitkäse,tortellini cheese
1352,27100,/rezept/259314/FLEISCH-Osso-Buco-a-la-Emi.html,kalbshaxe in dicken scheiben,kalbshaxe in dicken scheiben,veal
1353,61424,/rezept/78521/3-C-Suppe.html,chili con carne instant z.b.knorr,Chili con carne,Chili_con_carne
1354,26194,/rezept/264389/Berliner-Brot.html,mit schale gemahlene mandeln,mit schale gemahlene mandeln,almonds shell
1355,26194,/rezept/264389/Berliner-Brot.html,ein paar tropfen warmeswasser,ein paar tropfen warmeswasser,warm water


In [24]:
del_two_ingr=['baking dish','beaten stiff','tripe butcher','travel mittomatenmark','towards fibers','torties children',
'steamer insert','splits mix', 'dish cooker','toast days','green red']

In [25]:
two_ingr_pending=two_ingr_pending[~two_ingr_pending['en_ingr'].isin(del_two_ingr)]
two_ingr_pending.shape

(1344, 5)

In [26]:
pending_ingr=list(set(two_ingr_pending['en_ingr'].values.tolist()))
len(pending_ingr)

716

In [27]:
pending_ingr.sort()
pending_ingr

['',
 'Bouquet_garni',
 'Bratwurst',
 'Brunch_paprika',
 'Butterstreusel',
 'Chili_con_carne',
 'Herbes_de_Provence',
 'Metzgerzwiebel',
 'Noilly_Prat',
 'Nürnberger_Bratwurst',
 'Onke_Stichfest',
 'Patros_mit_Kräutern',
 'Pilzkraut',
 'Speckknödel',
 'alcohol',
 'almond flakes',
 'almond flour',
 'almond leaves',
 'almonds',
 'almonds seeberger',
 'almonds shell',
 'almonds skin',
 'alsoden juice',
 'anchovies oil',
 'anchovy',
 'apples boskop',
 'arab flatbread',
 'asparagus green',
 'asparagus spears',
 'asparagus white',
 'ausbasmati',
 'baby bell',
 'backkakao kaba',
 'bacon bacon',
 'bacon raw',
 'bacon rind',
 'baguettes bake',
 'baiser ',
 'ball mouse',
 'balsamic cream',
 'balsamic red',
 'balsamic vinegar',
 'banana',
 'basil leaves',
 'bass net',
 'batter schlutzkrapfen',
 'bean vegetables',
 'beans green',
 'beans white',
 'beef butcher',
 'beef hip',
 'beef juice',
 'beef ladder',
 'beef meat',
 'beef rib',
 'beef roast',
 'beer',
 'beetroot',
 'beetroot balls',
 'beetroot

In [ ]:
# balsamic cream-->crema di balsamico, ball mouse-->marshmallow, batter schlutzkrapfen-->Mezzelune,
# beef juice, beef ladder, beef meat, beef rib --> beef,biskin special-->Biskin_Spezial,
# bollgnesezb knorr, bolognese nudel-->Bolognese, brown cabbage-->collard, butter schnitze-->butter,
# candy sugar-->candy, carrots leek-->carrot, leek,champions heads-->mushroom
# chanterelle ragout-->pfifferlingsragout,chanterelles brine-->chanterelle, cheese sauce,cheese toppings-->cheese,
# chiliflocken-->chili,chili rot,chilli green-->chili pepper,zickelfleisch keule-->beef, yellow paprika-->yellow pepper,
# witch bolete-->Boletus erythropus,weisser pfefferadmühle-->white pepper,tuna oil-->tuna,

In [28]:
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('balsamic cream','Crema_di_balsamico'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('ball mouse','marshmallow'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('batter schlutzkrapfen','Mezzelune'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('beef juice','beef'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('beef ladder','beef'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('beef meat','beef'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('beef rib','beef'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('zickelfleisch keule','beef'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('biskin special','Biskin_Spezial'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('bollgnesezb knorr','Bolognese'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('bolognese nudel','Bolognese'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('brown cabbage','collard'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('butter schnitze','butter'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('candy sugar','candy'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('carrots leek','carrot,leek'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('champions heads','mushroom'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('chanterelle ragout','pfifferlingsragout'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('chanterelles brine','chanterelle'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('cheese sauce','cheese'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('cheese toppings','cheese'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('chiliflocken','chili'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('chili rot','chili_pepper'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('chilli green','chili_pepper'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('yellow paprika','yellow_pepper'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('witch bolete','Boletus_erythropus'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('weisser pfefferadmühle','white pepper'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('tuna oil','tuna'))

In [29]:
two_ingr_pending.shape

(1344, 5)

In [30]:
two_ingr_done_003=two_ingr_pending[two_ingr_pending['en_ingr'].str.split().str.len()==1]
two_ingr_done_003.shape

(293, 5)

In [31]:
two_ingr_done=pd.concat([two_ingr_done,two_ingr_done_003],axis=0,ignore_index=0)
two_ingr_done.shape

(893, 5)

In [37]:
two_ingr_done

,recipe_id,recipe_href,name,ingredient_name,en_ingr
2700,11059,/rezept/130203/Dr-Bello-s-Frikadellos.html,pfeffer aus der mühle weiss,weiss Pfeffer,white pepper
2701,2761,/rezept/478153/Auberginenroellchen-mit-Kaese.html,evtl. schwarze oliven zumgarnieren,schwarze oliven,black olives
2709,57419,/rezept/466738/Thymian-Zitronen-Butter.html,pfeffer aus der mühle weiss,weiss Pfeffer,white pepper
2714,117655,/rezept/352881/UEberbackene-Broetchen.html,pfeffer aus der mühle weiss,weiss Pfeffer,white pepper
2715,69626,/rezept/78736/Lauchspaetzle.html,pfeffer aus der mühle weiss,weiss Pfeffer,white pepper
...,...,...,...,...,...
1348,2396,/rezept/453198/Hirschbraten-mit-Pfifferlingen....,pfifferlinge eingelegt insalzlake siehe kb,pfifferlinge eingelegt insalzlake siehe kb,chanterelle
1350,104968,/rezept/489228/Salat-Cedro-mit-Bueffelmozzarel...,"wer das nicht hat,nimmt eingutes salz mit kräu...",pfeffer für rose habe ichkeine alternative,pepper
1352,27100,/rezept/259314/FLEISCH-Osso-Buco-a-la-Emi.html,kalbshaxe in dicken scheiben,kalbshaxe in dicken scheiben,veal
1353,61424,/rezept/78521/3-C-Suppe.html,chili con carne instant z.b.knorr,Chili con carne,Chili_con_carne


In [32]:
two_ingr_pending=two_ingr_pending[two_ingr_pending['en_ingr'].str.split().str.len()>1]
two_ingr_pending.shape

(1042, 5)

In [33]:
two_ingr_pending.reset_index(drop=True,inplace=True)

In [34]:
two_ingr_pending

,recipe_id,recipe_href,name,ingredient_name,en_ingr
0,15844,/rezept/392070/Fleisch-Rind-Koefte-an-Zitronen...,chilischoten piri-pirigetrocknet im mörserzers...,Peperoncino,chili pepper
1,95267,/rezept/385290/Makkaroni-Spinat-Auflauf.html,tk - würzspinat - minis,würzspinat,seasoned spinach
2,60116,/rezept/495673/Fleisch-Halsbraten-aus-dem-Roem...,"frische kräuter - hierrosmarin,maggikraut,oreg...",hierrosmarin,marin irons
3,29211,/rezept/418272/Caipi-Plaetzchen.html,cachaca - pitu o.ä,pitu o.ä,pitu oä
4,22430,/rezept/140128/Pilzrisotto-mit-schnell-gebrate...,cremechampignons - die mitden dunklen köpfen,cremechampignons,cream mushrooms
...,...,...,...,...,...
1037,110119,/rezept/436601/Knoblauchcremesuppe.html,tomaten getrocknet in öl,tomaten getrocknet in öl,tomatoes oil
1038,54577,/rezept/431309/Haehnchen-und-Spargel-in-Pesto-...,spargel weiss frisch d,spargel weiss frisch d,asparagus white
1039,95463,/rezept/257576/Pasta-Tortellini-mit-Weisswein-...,tortellini gefüllt mitkäse - wenn´s eilig ist ...,tortellini gefüllt mitkäse,tortellini cheese
1040,26194,/rezept/264389/Berliner-Brot.html,mit schale gemahlene mandeln,mit schale gemahlene mandeln,almonds shell


In [68]:
pending_ingr=list(set(two_ingr_pending['en_ingr'].values.tolist()))
len(pending_ingr)

355

In [69]:
pending_ingr.sort()
pending_ingr

['',
 'Blueberry',
 'Blumenkohl_Sud',
 'Bohnenstrudel',
 'Butter_aroma',
 'Camembert_cheese',
 'Cocktailtomato',
 'Curry_paste',
 'Foxys_vampirpaste',
 'Heichelheimer_Rohe_Kartoffelmasse',
 'Hirvenpotku',
 'Hörnchennudel',
 'Kräuter_der_Provence',
 'Kräuterbutter',
 'Kräutersalz',
 'Lollo_rosso',
 'Loup_de_mer',
 'Mettwurst',
 'Nusskruste',
 'Pesto_alla_Genovese',
 'Pfanni_Knödel',
 'Pimientos',
 'Pizza_e_Pasta',
 'Rama_Culinesse',
 'Ras_el_hanout',
 'Rot_Pfeffer',
 'Solperfleisch',
 'Sossenpulver',
 'Tafelspitz',
 'Tatare,knoblauch',
 'Tomato_al_gusto',
 'Vanille_Sossenpulver',
 'Würzcreme',
 'Würzspinat',
 'Zitronigfruchtiges_Curry',
 'alcohol',
 'almond',
 'almond_flour',
 'almonds',
 'apple',
 'asparagus',
 'babybel',
 'backkakao',
 'bacon',
 'baguettes',
 'balsamico',
 'basil',
 'bean',
 'beans',
 'beef',
 'beer',
 'beetroot',
 'beets',
 'bell_pepper',
 'berries',
 'biscuit',
 'bisquick',
 'bitter almonds',
 'bitter chocolate',
 'black_olives',
 'blackberry',
 'bread',
 'broccoli'

In [61]:
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('cremefine','creme'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('emmentalerodcomté gruyere','cheese'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('geleepulver kirsch','cherry_gelee_powder'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('greek',''))

two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('red peppers','red_bell_pepper'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('redfish cod','cod'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('tomato gustokräuter','Tomato_al_gusto'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('tomato gustokräuter','Tomato_al_gusto'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('tonkabohnenstrudelmit mangoragout','Bohnenstrudel'))

In [36]:
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('salt flower','salt'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('salt mortar','salt'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('salt sauce','salt'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('sambal olek','sambal oelek'))

In [49]:
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('almond flour','almond_flour'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('almonds shell','almond_flour'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('almonds skin','almond_flour'))

two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('baby bell','babybel'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('bean vegetables','bean'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('bisquick mix','bisquick'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('black halibut','halibut'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('butterl','butter'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('carrots carrots','carrots'))

In [ ]:
# herbs

In [37]:
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('herbs basil','basil'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('herbs chives','chives'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('herbs dill','dill'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('herbs esbasilikum','basil'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('herbs parsley','parsley'))

two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('l milk','milk'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('leek white','leek'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('meatloaf spices','meatloaf'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('sossen pulvervanille','Vanille_Sossenpulver'))

two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('sourdough flour','sourdough'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('soup vegetables','soup green'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('spicy peppers','chili'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('spicy woodpecker','seasoning'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('stadtwurst','sausage'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('steinbeisser','fish'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('sweet corn','corn'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('toast rind','toasted_rind'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('tomato kräuter','toasted_rind'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('vegetable palmin','vegetable_oil'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('white cabbage','cabbage'))

In [ ]:
# split with ','
# 'gusto herbs','hazelnuts sugar','olives peppers','oregano garlic','parsley dille','pepper grinder','pepper salt',
# 'pepperoni oil','pity casting','pralinenform für','pulver bdroetker','raisins rum','salt pepper','scampis herbs',
# 'thyme rosemary','tomatoes oregano','cornstarch cream','cumin potatoes','ginger garlic','ginger walnut',
# 'herbs salt','kool whip','meat tastes','mitmiracle seesaw','olives peppers','paprika cabanossi','pralinenform für',
# 'pulver bdroetker', 'scampis herbs','tomato basil','tomatoes garlic','tomatoes oregano',

In [38]:
split_list=['gusto herbs','hazelnuts sugar','olives peppers','oregano garlic','parsley dille','pepper grinder','pepper salt',
'pepperoni oil','pity casting','pralinenform für','pulver bdroetker','raisins rum','salt pepper','scampis herbs',
'thyme rosemary','tomatoes oregano','cornstarch cream','cumin potatoes','ginger garlic','ginger walnut',
'herbs salt','kool whip','meat tastes','mitmiracle seesaw','olives peppers','paprika cabanossi','pralinenform für',
'pulver bdroetker', 'scampis herbs','tomato basil','tomatoes garlic','tomatoes oregano']

In [39]:
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace(' ',',') if x in split_list else x)

In [ ]:
# wrong translation

In [40]:
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='hierrosmarin']='rosemary'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='kokosmasse']='Kokosmasse'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='hörnchen teigware']='Hörnchennudel'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='hirvenpotku']='Hirvenpotku'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='lebkuchenherz']='Lebkuchenherz'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='heichelheimer kartoffelmasseroh']='Heichelheimer_Rohe_Kartoffelmasse'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='kräuterbutter']='Kräuterbutter'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='kräuteröl']='Kräuteröl'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='teel zitronigfruchtiges curry']='Zitronigfruchtiges_Curry'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='2limettenscheiben reintun']='lime'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='flüssigkeit von dengewürzgurken']='Liquid_from_the_pickled_cucumbers'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='messerspitze raz el hanut']='Ras_el_hanout'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='nusskruste']='Nusskruste'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='pimientos de padrón']='Pimientos'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='kleine dunklefladen mit anis']='flatbread'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='pesto alla genovese stehtim kb']='Pesto_alla_Genovese'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='pfanni knödel halb halb']='Pfanni_Knödel'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='pizza epasta']='Pizza_e_Pasta'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='rama sahne']='cream'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='ras el hanout']='Ras_el_hanout'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='lollo rosso']='Lollo_rosso'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='rot Pfeffer']='Rot_Pfeffer'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='paprikaschote rot']='red_bell_pepper'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='paprika rot']='red_bell_pepper'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='rote paprikastreifen']='red_bell_pepper'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='halbe rote paprikaschotesehr fein gewürfelt']='red_bell_pepper'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='paprika rot frisch instreifen']='red_bell_pepper'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='von der roten spitzpaprika']='chili'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='mettwürste o. knacker inscheiben geschnitten']='Mettwurst'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='kräutersalz']='Kräutersalz'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='würzspinat']='Würzspinat'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='würzcreme']='Würzcreme'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='scheinelenden']='pork'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='solperfleisch']='Solperfleisch'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='sossenpulver']='Sossenpulver'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='sossen pulvervanille']='Sossenpulver_vanille'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='sud vomblumenkohl']='Blumenkohl_Sud'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='ein paar spritzer maggi']='spritzer'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='fichtenreizker']='mushroom'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='le tatare mit knoblauch']='Tatare,knoblauch'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='loup de mer filet']='Loup_de_mer'

two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='cocktailtomaten in der dose']='Cocktailtomato'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='kroketten für den backofen']='carrot'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='rama culinesse']='Rama_Culinesse'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='wenige tropfen aroma%x2cbutter']='Butter_aroma'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='mehlige = ungefaehr 2pf']='flour'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='paprika edelsüss']='paprika_powder'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='pirigetrocknet im mörserzerstossen']='chili_powder'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='sambal olek']='sambal oelek'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='tomate al gusto kräuter']='Tomato_al_gusto'
two_ingr_pending['en_ingr'][two_ingr_pending['ingredient_name']=='tomato al gusto mitkräutern']='Tomato_al_gusto'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [252]:
two_ingr_pending[two_ingr_pending['en_ingr']=='baby bell']['ingredient_name'].values

array(['babybel'], dtype=object)

In [ ]:
# totally delete
# 'glasses weck','glasses á','great lotte','green oblique','green side','kool whip'

In [41]:
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('glasses weck',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('glasses á',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('great lotte',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('green oblique',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('green side',''))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('kool whip',''))

In [66]:
del_ingrs=['seeberger','alsoden','arab','kaba','bake','vinegar','butcher','biber','bottom','florets','barked','bowl',
         'brown', 'bunny','grate','chinese','whipped','turns','fillet','fillets','medium','need','clove','cloves',
         'washed','halves','grinder','separate','smoked','spelled','clover','cloves','clove','rama','oven','double',
         'fillets','fillet', 'finess','food','frankfurter','blue','grilled','hearts','iglo','nachgusto', 'sprinkle',
         'ige','indian','ingredient','instant','italian','ribs','keback','manufacturer','knorr','washed','leaf','leaves',
         'aroma','abrasion','maggi','mania','set','marrow','filling','bavarian','nesquik','neuteral','neutral','cutresistant',
         'back','filet','flakes','leaf','bush','kernels','allspice','shots','chest','pink','oä','boneless','hand','fin',
         'fillings','replacement','rib','ribs','roasted','sprigs','harvest','kind','spelled','spicy woodpecker',
         'blup','teel','thuringian','toe','must',
        'selfmade','rolls','cooking','substitute','firm','marinade','harzer', 'hungary','jacket','fill','pickles',
         'crêpes','rzin', 'serving','shortcrust','shot','fox','kato','trip','wort', 'pods','yellow','flavor','swiss',
          'shell','rote','red']

In [67]:
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:del_ingr(x))

In [ ]:
# 'seeberger','alsoden','arab','kaba','bake','vinegar','butcher','biber','bottom','florets','barked','bowl',
# 'brown', 'bunny','grate','chinese','whipped','turns','fillet',','fillets','medium', 'need','clove','cloves',
# 'washed','halves','grinder','separate','smoked','spelled','clover','cloves','clove','rama','oven','double',
# 'fillets','fillet', 'finess','food','frankfurter','blue','grilled','hearts','iglo','nachgusto', 'sprinkle',
# 'ige','indian','ingredient','instant','italian','ribs','keback','manufacturer','knorr','washed','leaf','leaves',
# 'aroma','abrasion','maggi','mania','set','marrow','filling','bavarian','nesquik','neuteral','neutral','cutresistant'
# 'back','filet','flakes','leaf','bush','kernels','allspice','shots','chest','pink','oä','boneless','hand','fin',
# 'fillings','replacement','rib','ribs','roasted','sprigs','harvest','kind','spelled','spicy woodpecker',
# 'blup','teel','thuringian','toe','must'

# 'selfmade','rolls','cooking','substitute','firm','marinade','harzer', 'hungary','jacket','fill','pickles',
# 'crêpes','rzin', 'serving','shortcrust','shot','fox','kato','trip','wort', 'pods','yellow','flavor'

In [ ]:
# if contain these, then just keep these in that cell
# asparagus,bacon,beef,celery,cereal,chicken, chickpeas,chickweed, zucchini, water, venison,tortellini,kohlrabi,
# margarine,mushrooms,onion，

In [55]:
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('asparagus'),'en_ingr']='asparagus'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('bacon'),'en_ingr']='bacon'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('beef'),'en_ingr']='beef'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('celery'),'en_ingr']='celery'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('cereal'),'en_ingr']='cereal'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('chicken'),'en_ingr']='chicken'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('chickpeas'),'en_ingr']='chickpeas'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('chickweed'),'en_ingr']='chickweed'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('zucchini'),'en_ingr']='zucchini'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('water'),'en_ingr']='water'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('venison'),'en_ingr']='venison'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('tortellini'),'en_ingr']='tortellini'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('kohlrabi'),'en_ingr']='kohlrabi'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('margarine'),'en_ingr']='margarine'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('mushrooms'),'en_ingr']='mushrooms'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('onion'),'en_ingr']='onion'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('balsamic'),'en_ingr']='balsamico'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('flor sal'),'en_ingr']='salt'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('boskop'),'en_ingr']='apple'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('backkakao'),'en_ingr']='backkakao'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('beetroot'),'en_ingr']='beetroot'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('chilipaste'),'en_ingr']='chili_paste'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('cinnamon'),'en_ingr']='cinnamon'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('coffee'),'en_ingr']='coffee'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('couverture'),'en_ingr']='couverture'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('creamfine'),'en_ingr']='cream'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('gelatin'),'en_ingr']='gelatin'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('dinkelbrot'),'en_ingr']='bread'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('hanout'),'en_ingr']='Ras_el_hanout'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('herbs provence'),'en_ingr']='Kräuter_der_Provence'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('rosemary'),'en_ingr']='rosemary'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('tragon'),'en_ingr']='tarragon'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('domina'),'en_ingr']='grape'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('horseradish'),'en_ingr']='horseradish'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('lettuce'),'en_ingr']='lettuce'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('camembert'),'en_ingr']='Camembert_cheese'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('jalapeño'),'en_ingr']='jalapeno'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('kaktusfeige'),'en_ingr']='prickly_pear'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('jerky'),'en_ingr']='beef'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('cucumbers'),'en_ingr']='cucumber'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('beer'),'en_ingr']='beer'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('kormacurrypaste'),'en_ingr']='Curry_paste'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('mozzarella'),'en_ingr']='mozzarella_cheese'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('parmesan'),'en_ingr']='parmesan_cheese'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('mushroom'),'en_ingr']='mushroom'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('oil'),'en_ingr']='oil'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('pitahayas'),'en_ingr']='pitahaya'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('pork'),'en_ingr']='pork'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('rahbarbar'),'en_ingr']='rahbarbar'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('rice'),'en_ingr']='rice'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('sardines'),'en_ingr']='sardines'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('sauerkraut'),'en_ingr']='sauerkraut'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('tabasco'),'en_ingr']='tabasco'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('sugar'),'en_ingr']='sugar'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('tafelspitz'),'en_ingr']='Tafelspitz'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('duck'),'en_ingr']='duck'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('crabs'),'en_ingr']='crab'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('yeast'),'en_ingr']='yeast'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('pernod'),'en_ingr']='alcohol'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('schmalz'),'en_ingr']='lard'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('nutella'),'en_ingr']='nutella'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('nutmeg'),'en_ingr']='nutmeg'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('shrimp'),'en_ingr']='shrimp'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('noodles'),'en_ingr']='noodle'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('trout'),'en_ingr']='trout'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('tortellini'),'en_ingr']='tortellini'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('tuna'),'en_ingr']='tuna'
two_ingr_pending.loc[two_ingr_pending['en_ingr'].str.contains('turkey'),'en_ingr']='turkey'

In [56]:
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('tuna oil','tuna'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('ausbasmati','rice'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('flower salt','salt'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('foxys vampirpaste','Foxys_vampirpaste'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('fruit blueberries','Blueberry'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('gelatine sheets','gelatin'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('green paprika','green_bell_pepper'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('greenroasted pepper','green_bell_pepper'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('herbs ramsons','ramsons'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('hot peppers','chili'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('kalahari salt','salt'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('kidnej beans','kidney_bean'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('kipfler solid','potato'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('knead flour','flour'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('knight sport','chocolate'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('lump sugar','sugar'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('löwensenf sausages','senf,sausage'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('morello cherries','sour_cherry'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('oil tomatoes','oil_tomato'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('pizza tomatoes','tomato'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('roll rolo','Lollo_rosso'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('savoy cabbage','wirsing'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('sweet peppers','bell_pepper'))

two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('chili pepper','chili'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('chili peppers','chili'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('creamfine','cream'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('cremfine branch','cream'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('emmentalerodcomté gruyere','cheese'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('grana pandano','cheese'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('mustard sweet','sweet_mustard'))

In [57]:
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('zest lime','lime_zest'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('zest lemon','lemon_zest'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('beans green','green_bean'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('beans white','white_bean'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('beef roast','roasted_beef'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('cheese gouda','gouda_cheese'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('cheese hattegouda','gouda_cheese'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('cheese parmesan','parmesan_cheese'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('cream vanilla','vanilla_cream'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('cremepuddingpulver vanille','vanilla_powder'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('cremepulver vanille','vanilla_powder'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('powder vanilla','vanilla_powder'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('pudding vanilla','vanilla_powder'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('puddingpulver vanillegesckmack','vanilla_powder'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('liver pig','pig_liver'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('muffins peaches','peach_muffin'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('olives black','black_olives'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('pepper red','red_pepper'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('peppers red','red_pepper'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('pepper white','white_pepper'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('powder raspberry','raspberry_powder'))


two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('juice peaches','peach_juice'))
two_ingr_pending['en_ingr']=two_ingr_pending['en_ingr'].apply(lambda x:x.replace('chocolate white','white_chocolate'))

In [46]:
# change order
# zest lime,zest lemon,beans green, beans white,beef roast,cheese gouda,cheese hattegouda,cheese parmesan,
# cream vanilla, cremepuddingpulver vanille,cremepulver vanille,liver pig,muffins peaches, olives black,
# pepper red,pepper white,peppers red,powder raspberry, powder vanilla, pudding vanilla,puddingpulver vanillegesckmack
# rib pig,ribs pig, 

In [70]:
two_ingr_done

,recipe_id,recipe_href,name,ingredient_name,en_ingr
2700,11059,/rezept/130203/Dr-Bello-s-Frikadellos.html,pfeffer aus der mühle weiss,weiss Pfeffer,white pepper
2701,2761,/rezept/478153/Auberginenroellchen-mit-Kaese.html,evtl. schwarze oliven zumgarnieren,schwarze oliven,black olives
2709,57419,/rezept/466738/Thymian-Zitronen-Butter.html,pfeffer aus der mühle weiss,weiss Pfeffer,white pepper
2714,117655,/rezept/352881/UEberbackene-Broetchen.html,pfeffer aus der mühle weiss,weiss Pfeffer,white pepper
2715,69626,/rezept/78736/Lauchspaetzle.html,pfeffer aus der mühle weiss,weiss Pfeffer,white pepper
...,...,...,...,...,...
1339,70819,/rezept/447113/Schnelles-Thunfischbroetchen-al...,thunfisch konserve in ölabgetropft,thunfisch konserve in ölabgetropft,tuna
1348,2396,/rezept/453198/Hirschbraten-mit-Pfifferlingen....,pfifferlinge eingelegt insalzlake siehe kb,pfifferlinge eingelegt insalzlake siehe kb,chanterelle
1350,104968,/rezept/489228/Salat-Cedro-mit-Bueffelmozzarel...,"wer das nicht hat,nimmt eingutes salz mit kräu...",pfeffer für rose habe ichkeine alternative,pepper
1352,27100,/rezept/259314/FLEISCH-Osso-Buco-a-la-Emi.html,kalbshaxe in dicken scheiben,kalbshaxe in dicken scheiben,veal


In [ ]:
# concat the pending data and done data, re-select the pending data

In [71]:
two_ingr = pd.concat([two_ingr_done,two_ingr_pending],axis=0,ignore_index=True)
two_ingr.shape

(1935, 5)

In [80]:
two_ingr.tail()

,recipe_id,recipe_href,name,ingredient_name,en_ingr
1979,110119,/rezept/436601/Knoblauchcremesuppe.html,tomaten getrocknet in öl,tomaten getrocknet in öl,oil
1980,54577,/rezept/431309/Haehnchen-und-Spargel-in-Pesto-...,spargel weiss frisch d,spargel weiss frisch d,asparagus
1981,95463,/rezept/257576/Pasta-Tortellini-mit-Weisswein-...,tortellini gefüllt mitkäse - wenn´s eilig ist ...,tortellini gefüllt mitkäse,tortellini
1982,26194,/rezept/264389/Berliner-Brot.html,mit schale gemahlene mandeln,mit schale gemahlene mandeln,almond_flour
1983,26194,/rezept/264389/Berliner-Brot.html,ein paar tropfen warmeswasser,ein paar tropfen warmeswasser,water


In [ ]:
# explode columns with ','

In [79]:
two_ingr=two_ingr.assign(en_ingr=two_ingr['en_ingr'].str.split(',')).explode('en_ingr') #https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows
two_ingr.reset_index(drop=True,inplace=True)
two_ingr.shape

(1984, 5)

In [81]:
# select done data: lengthn==1
done_ingr_data=two_ingr[two_ingr['en_ingr'].str.split().str.len()==1]
done_ingr_data.shape

(1243, 5)

In [84]:
lemmatizer = WordNetLemmatizer()

In [86]:
done_ingr_data['lemma_ingr']=done_ingr_data['en_ingr'].apply(lambda x:lemmatizer.lemmatize(x))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [87]:
done_ingr_data.head()

,recipe_id,recipe_href,name,ingredient_name,en_ingr,lemma_ingr
599,74301,/rezept/381870/Nudelsalat.html,verschiedenes gemüse ausdem glas,verschiedenes gemüse ausdem glas,vegetables,vegetable
600,55182,/rezept/404267/Krabbenruehrei.html,pfeffer aus der mühle bunt,bunt Pfeffer,pepper,pepper
601,94189,/rezept/351602/Hokaido-Eintopf.html,chilli rot grob gehackt,Chili rot,chili_pepper,chili_pepper
602,101709,/rezept/382419/karamellisierte-Feigen-mit-Zieg...,etwas crema di balsamico,crema di balsamico,Crema_di_balsamico,Crema_di_balsamico
603,68217,/rezept/260059/Penne-Rigate-mit-Hackfleisch.html,"chiliflocken rot - abervorsicht,die machenwirk...",chiliflocken rot,chili_pepper,chili_pepper


In [98]:
pending_ingr=two_ingr[two_ingr['en_ingr'].str.split().str.len()!=1]
pending_ingr=pending_ingr[pending_ingr['en_ingr']!='']
pending_ingr.dropna(inplace=True)
pending_ingr.reset_index(drop=True,inplace=True)
pending_ingr.shape

(732, 5)

In [106]:
pending_ingr['lemma_ingr']=pending_ingr['en_ingr'].apply(lambda x:x.split()[0])+'_'+pending_ingr['en_ingr'].apply(lambda x:lemmatizer.lemmatize(x.split()[1]))

In [107]:
pending_ingr.head()

,recipe_id,recipe_href,name,ingredient_name,en_ingr,lemma_ingr
0,11059,/rezept/130203/Dr-Bello-s-Frikadellos.html,pfeffer aus der mühle weiss,weiss Pfeffer,white pepper,white_pepper
1,2761,/rezept/478153/Auberginenroellchen-mit-Kaese.html,evtl. schwarze oliven zumgarnieren,schwarze oliven,black olives,black_olive
2,57419,/rezept/466738/Thymian-Zitronen-Butter.html,pfeffer aus der mühle weiss,weiss Pfeffer,white pepper,white_pepper
3,117655,/rezept/352881/UEberbackene-Broetchen.html,pfeffer aus der mühle weiss,weiss Pfeffer,white pepper,white_pepper
4,69626,/rezept/78736/Lauchspaetzle.html,pfeffer aus der mühle weiss,weiss Pfeffer,white pepper,white_pepper


In [108]:
env_two_ingr=pd.concat([done_ingr_data,pending_ingr],axis=0,ignore_index=True)
env_two_ingr.shape

(1975, 6)

In [110]:
env_two_ingrs=list(set(env_two_ingr['lemma_ingr'].values.tolist()))

In [111]:
len(env_two_ingrs)

470

In [112]:
env_two_ingrs.sort()
env_two_ingrs

['Biskin_Spezial',
 'Blueberry',
 'Blumenkohl_Sud',
 'Bohnenstrudel',
 'Boletus_erythropus',
 'Bolognese',
 'Bouquet_garni',
 'Bratwurst',
 'Brunch_paprika',
 'Butter_aroma',
 'Butterstreusel',
 'Camembert_cheese',
 'Chili_con_carne',
 'Cocktailtomato',
 'Crema_di_balsamico',
 'Curry_paste',
 'Foxys_vampirpaste',
 'Heichelheimer_Rohe_Kartoffelmasse',
 'Herbes_de_Provence',
 'Hirvenpotku',
 'Hörnchennudel',
 'Kräuter_der_Provence',
 'Kräuterbutter',
 'Kräutersalz',
 'Lollo_rosso',
 'Loup_de_mer',
 'Mettwurst',
 'Metzgerzwiebel',
 'Mezzelune',
 'Noilly_Prat',
 'Nusskruste',
 'Nürnberger_Bratwurst',
 'Onke_Stichfest',
 'Patros_mit_Kräutern',
 'Pesto_alla_Genovese',
 'Pfanni_Knödel',
 'Pilzkraut',
 'Pimientos',
 'Pizza_e_Pasta',
 'Rama_Culinesse',
 'Ras_el_hanout',
 'Rot_Pfeffer',
 'Solperfleisch',
 'Sossenpulver',
 'Speckknödel',
 'Tafelspitz',
 'Tatare',
 'Tomato_al_gusto',
 'Vanille_Sossenpulver',
 'Würzcreme',
 'Würzspinat',
 'Zitronigfruchtiges_Curry',
 'alba_oil',
 'alcohol',
 'almon

In [113]:
env_two_ingr['lemma_ingr'][env_two_ingr['lemma_ingr']=='tomatoes_juice']='tomato_juice'
env_two_ingr['lemma_ingr'][env_two_ingr['lemma_ingr']=='tomato_mitkräutern']='tomato_with_herbs'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [114]:
env_two_ingr

,recipe_id,recipe_href,name,ingredient_name,en_ingr,lemma_ingr
0,74301,/rezept/381870/Nudelsalat.html,verschiedenes gemüse ausdem glas,verschiedenes gemüse ausdem glas,vegetables,vegetable
1,55182,/rezept/404267/Krabbenruehrei.html,pfeffer aus der mühle bunt,bunt Pfeffer,pepper,pepper
2,94189,/rezept/351602/Hokaido-Eintopf.html,chilli rot grob gehackt,Chili rot,chili_pepper,chili_pepper
3,101709,/rezept/382419/karamellisierte-Feigen-mit-Zieg...,etwas crema di balsamico,crema di balsamico,Crema_di_balsamico,Crema_di_balsamico
4,68217,/rezept/260059/Penne-Rigate-mit-Hackfleisch.html,"chiliflocken rot - abervorsicht,die machenwirk...",chiliflocken rot,chili_pepper,chili_pepper
...,...,...,...,...,...,...
1970,52457,/rezept/413272/Doener-Topf.html,döner gewürz für diemühle,döner gewürz für diemühle,doner spice,doner_spice
1971,96968,/rezept/90568/Schneckenposts-Ravioli-mit-Parma...,teig nach meinemrezept %x22pasta-gr,teig nach meinemrezept %x22pasta,dough pasta,dough_pasta
1972,64233,/rezept/382128/Bandnudeln-mit-Rucola-Tomaten.html,"salz,weisser pfeffera.d.mühle am besten",weisser pfeffera.d.mühle am besten,white pepper,white_pepper
1973,57513,/rezept/404896/Bolognese-Gratin-auf-meine-Art....,tomate al gusto mitkräutern,tomate al gusto mitkräutern,tomato mitkräutern,tomato_with_herbs


In [115]:
env_two_ingr.to_csv('~/work/food_flavor/kochbar_ingredient_preprocessing/datasets/long_ingr_data/two_ingr_data/two_ingr_done.csv')